In [35]:
import shutil

source_file_path = '.\..\.env'

destination_file_path = '.'

shutil.copy(source_file_path, destination_file_path)

'.\\.env'

In [36]:
from datetime import datetime 
import pandas as pd
import numpy as np
from dotenv import dotenv_values
from bingx.api import BingxAPI

config = dotenv_values(".env")


API_KEY = config["API_KEY"]
SECRET_KEY = config["SECRET_KEY"]

bingx =  BingxAPI(API_KEY, SECRET_KEY, timestamp="local")

In [37]:
# RSI chatgpt (not work)

def calculate_rsi(data, period=14):
    df = pd.DataFrame(data)
    df['close'] = pd.to_numeric(df['close'], errors='coerce')
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)
    
    # Adjust the rolling mean calculation for small sample sizes
    avg_gain = gain.rolling(window=min(len(gain), period), min_periods=1).mean()
    avg_loss = loss.rolling(window=min(len(loss), period), min_periods=1).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Example data

res = []
data = bingx.get_kline_data("BTC-USDT","1h" ,1716478063000 ,1719156463000, 1440)
res += list(reversed(data))

rsi_values = calculate_rsi(res)
print(rsi_values)



0            NaN
1       0.000000
2       0.000000
3       0.000000
4      48.897747
         ...    
740    54.159825
741    52.857906
742    28.193219
743    24.904267
744    16.299148
Name: close, Length: 745, dtype: float64


In [38]:
# RSI https://github.com/lukaszbinden/rsi_tradingview/blob/main/rsi.py (Worked!! Nice)
def rsi_tradingview(ohlc: pd.DataFrame, period: int = 14, round_rsi: bool = True):
    """ Implements the RSI indicator as defined by TradingView on March 15, 2021.
        The TradingView code is as follows:
        //@version=4
        study(title="Relative Strength Index", shorttitle="RSI", format=format.price, precision=2, resolution="")
        len = input(14, minval=1, title="Length")
        src = input(close, "Source", type = input.source)
        up = rma(max(change(src), 0), len)
        down = rma(-min(change(src), 0), len)
        rsi = down == 0 ? 100 : up == 0 ? 0 : 100 - (100 / (1 + up / down))
        plot(rsi, "RSI", color=#8E1599)
        band1 = hline(70, "Upper Band", color=#C0C0C0)
        band0 = hline(30, "Lower Band", color=#C0C0C0)
        fill(band1, band0, color=#9915FF, transp=90, title="Background")

    :param ohlc:
    :param period:
    :param round_rsi:
    :return: an array with the RSI indicator values
    """
    df['close'] = pd.to_numeric(df['close'], errors='coerce')
    delta = ohlc["close"].diff()

    up = delta.copy()
    up[up < 0] = 0
    up = pd.Series.ewm(up, alpha=1/period).mean()

    down = delta.copy()
    down[down > 0] = 0
    down *= -1
    down = pd.Series.ewm(down, alpha=1/period).mean()

    rsi = np.where(up == 0, 0, np.where(down == 0, 100, 100 - (100 / (1 + up / down))))

    return np.round(rsi, 2) if round_rsi else rsi




start_timestamp = 1720688771000
# start_timestamp = 1704122863000
current_timestamp = int(datetime.now().timestamp() * 1000)
bool = True
res = []
while bool:
    data = list(bingx.get_kline_data("BTC-USDT","1h" ,start_timestamp ,limit= 1440))
    
    start_timestamp = data[0]['time'] + 1
    res += list(reversed(data))
    
    if len(data) < 1440: 
        break

df = pd.DataFrame(res)

# print(df.__len__())
df['close'] = pd.to_numeric(df['close'], errors='coerce')
array_list = rsi_tradingview(df)
b = array_list.tolist()
# convertToJsonFile(b, "rsi-BTC", ".") 
# convertToJsonFile(res, "BTC-ohlc", ".") 
# json.dump(b, codecs.open("test.json", 'w', encoding='utf-8'), 
#           separators=(',', ':'), 
#           sort_keys=True, 
#           indent=4)
# json.dump(res, codecs.open("test1.json", 'w', encoding='utf-8'), 
#           separators=(',', ':'), 
#           sort_keys=True, 
#           indent=4)
# 


